<img width="150" alt="Logo_ER10" src="https://user-images.githubusercontent.com/3244249/151994514-b584b984-a148-4ade-80ee-0f88b0aefa45.png">

### Interpreting the sentence classification model with LIME

LIME (Local Interpretable Model-agnostic Explanations) is an explainable-AI method that aims to create an interpretable model that locally represents the classifier. For more details see the [LIME paper](https://arxiv.org/abs/1602.04938).

Note that this notebook was adapted from the [LIME/text tutorial for DIANNA](https://github.com/dianna-ai/dianna/blob/main/tutorials/lime_text.ipynb).

#### Colab Setup

In [1]:
import pandas as pd

from classify_text_with_inlegal_bert_xgboost import classify_texts

running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
  # install dianna
  !python3 -m pip install dianna[notebooks]
  
  # download data used in this demo
  import os 
  base_url = 'https://raw.githubusercontent.com/dianna-ai/dianna/main/tutorials/'
  paths_to_download = ['data/movie_reviews_word_vectors.txt', 'models/movie_review_model.onnx']
  for path in paths_to_download:
      !wget {base_url + path} -P {os.path.dirname(path)}

#### 1. Imports and paths

In [2]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from torchtext.vocab import Vectors
from scipy.special import expit as sigmoid
from pathlib import Path

import dianna
from dianna import visualization
from dianna import utils
from dianna.utils.tokenizers import SpacyTokenizer
from train_inlegalbert_xgboost import class_names

In [3]:
model_path = Path('inlegal_bert_xgboost_classifier.json')

#### Some test data

In [4]:
constitutive_statement_0 = "The purchase, import or transport from Syria of crude oil and petroleum products shall be prohibited."
constitutive_statement_1 = "This Decision shall enter into force on the twentieth day following that of its publication in the Official Journal of the European Union."
regulatory_statement_0 = "Where observations are submitted, or where substantial new evidence is presented, the Council shall review its decision and inform the person or entity concerned accordingly."
regulatory_statement_1 = "The relevant Member State shall inform the other Member States of any authorisation granted under this Article."
regulatory_statement_2 = "Member States shall cooperate, in accordance with their national legislation, with inspections and disposals undertaken pursuant to paragraphs 1 and 2."

Loading the model



In [5]:
class StatementClassifier:
    def __init__(self):
        self.tokenizer = SpacyTokenizer(name='en_core_web_sm')

    def __call__(self, sentences):
        # ensure the input has a batch axis
        if isinstance(sentences, str):
            sentences = [sentences]

        probs = classify_texts(sentences, model_path, return_proba=True)

        return np.transpose([(probs[:, 0]), (1 - probs[:, 0])])
            

In [6]:
# define model runner. max_filter_size is a property of the model
model_runner = StatementClassifier()

#### Test the model

In [7]:
prediction = model_runner([constitutive_statement_0,constitutive_statement_1, regulatory_statement_0, regulatory_statement_1,regulatory_statement_2])
[class_names[m] for m in np.argmax(prediction, axis=1)]

Some weights of the model checkpoint at law-ai/InLegalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Creating features: 100%|██████████| 5/5 [00:00<00:00, 16.87it/s]


Loading model from inlegal_bert_xgboost_classifier.json.


['constitutive', 'constitutive', 'regulatory', 'regulatory', 'regulatory']

#### Set parameters for DIANNA

In [8]:
label_of_interest = 1
print('label_of_interest is', class_names[label_of_interest])
statement = regulatory_statement_0
num_samples = 1000
num_features=100  # top n number of words to include in the attribution map

def run_dianna(input_text):
    return dianna.explain_text(model_runner, input_text, model_runner.tokenizer,
                               'LIME', labels=[label_of_interest], num_samples=num_samples, num_features=num_features, )[0]

label_of_interest is regulatory


#### Are the results stable with current parameters?

In [9]:
explanation_relevances = [run_dianna(statement) for i in range(3)]
sorted_relevances = [sorted(r, key=lambda t : t[1]) for r in explanation_relevances]

pd.DataFrame([[r[2] for r in sr] for sr in sorted_relevances], columns=[r[0] for r in sorted_relevances[0]]).describe()

Some weights of the model checkpoint at law-ai/InLegalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Creating features: 100%|██████████| 1000/1000 [01:55<00:00,  8.66it/s]
Some weights of the model checkpoint at law-ai/InLe

,Where,observations,are,submitted,",",or,where,substantial,new,evidence,...,decision,and,inform,the,person,or,entity,concerned,accordingly,.
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
mean,-0.005973,0.033819,0.044202,0.050789,0.074258,0.064762,0.023754,0.034969,0.069220,0.082735,...,0.038275,0.096304,0.095057,0.063708,0.012272,0.058115,0.052506,0.082051,0.071340,0.049662
std,0.016238,0.006634,0.014064,0.016550,0.014820,0.028109,0.011242,0.024919,0.010647,0.004493,...,0.021006,0.024024,0.020876,0.029332,0.035683,0.013160,0.014422,0.013582,0.033703,0.010203
min,-0.023122,0.026857,0.028155,0.032984,0.057950,0.046526,0.016915,0.007785,0.057084,0.080027,...,0.015405,0.068844,0.071017,0.030647,-0.021165,0.044011,0.042107,0.066830,0.033687,0.038507
25%,-0.013544,0.030694,0.039109,0.043332,0.067935,0.048576,0.017266,0.024089,0.065336,0.080142,...,0.029058,0.087733,0.088275,0.052257,-0.006513,0.052140,0.044274,0.076609,0.057666,0.045231
50%,-0.003966,0.034531,0.050063,0.053681,0.077920,0.050626,0.017618,0.040393,0.073589,0.080257,...,0.042711,0.106622,0.105532,0.073868,0.008138,0.060269,0.046440,0.086389,0.081645,0.051956
75%,0.002601,0.037300,0.052225,0.059691,0.082412,0.073880,0.027174,0.048561,0.075288,0.084089,...,0.049710,0.110034,0.107076,0.080239,0.028990,0.065167,0.057705,0.089662,0.090166,0.055239
max,0.009168,0.040068,0.054388,0.065702,0.086903,0.097133,0.036729,0.056730,0.076988,0.087921,...,0.056709,0.113446,0.108620,0.086610,0.049842,0.070064,0.068970,0.092935,0.098686,0.058523


Seems quite stable with 1000 samples in LIME. We can now run DIANNA knowing results will contain mostly signal and not just noise.

In [10]:
explanation_relevance = run_dianna(statement)
print('attributions for class', class_names[label_of_interest])
pd.DataFrame(explanation_relevance)

Some weights of the model checkpoint at law-ai/InLegalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Creating features: 100%|██████████| 1000/1000 [01:57<00:00,  8.52it/s]

attributions for class regulatory


,0,1,2
0,inform,20,0.117588
1,",",12,0.099412
2,are,2,0.090272
3,accordingly,26,0.089739
4,.,27,0.089180
5,review,16,0.087251
6,evidence,9,0.082871
7,shall,15,0.081039
8,and,19,0.080405
9,its,17,0.078664


#### Visualize the result
DIANNA includes a visualization package, capable of highlighting the relevance of each word in the text for a chosen class. The visualization is in HTML format.
Words in favour of the selected class are highlighted in red, while words against the selected class - in blue.

In [11]:
visualization.highlight_text(explanation_relevance, model_runner.tokenizer.tokenize(statement))